In [127]:
from kendra_retriever_falcon_40b_instruct import run_chain, build_chain
from kendra_chat_falcon_40b_instruct import run_chain as run_chain_chat, build_chain as build_chain_chat
from langchain.prompts import PromptTemplate
from langchain.llms.sagemaker_endpoint import LLMContentHandler
from langchain import SagemakerEndpoint
from sentence_transformers import SentenceTransformer, util

from tqdm import tqdm
import pandas as pd
import json

import os
os.environ['AWS_REGION'] = 'us-east-1'
os.environ["KENDRA_INDEX_ID"] = '6bf94090-16d3-4c34-b541-2eaadb4fe5f1'
os.environ["FALCON_40B_INSTRUCT_ENDPOINT"] = 'falcon-40b-instruct-12xl'

In [128]:
def get_test_bank():
    test_bank = """Question: What is AWS Kendra?
    Answer: AWS Kendra is an intelligent search service powered by machine learning that provides natural language search capabilities for various data sources and content.

    Question: What types of data sources does AWS Kendra support for indexing?
    Answer: AWS Kendra supports various data sources, including Amazon S3, SharePoint Online, Salesforce, ServiceNow, Relational Databases, and more.

    Question: How does AWS Kendra handle natural language queries?
    Answer: AWS Kendra uses machine learning models to understand natural language queries and provide relevant results using contextual understanding and ranking.

    Question: What are the benefits of using AWS Kendra for enterprise search?
    Answer: Some benefits of AWS Kendra include improved search accuracy, reduced time to find relevant information, and support for multiple data sources.

    Question: How does AWS Kendra ensure security and compliance?
    Answer: AWS Kendra encrypts data at rest and in transit, provides access control via AWS Identity and Access Management (IAM), and is compliant with various security standards.

    Question: Can AWS Kendra be used for both internal and external search scenarios?
    Answer: Yes, AWS Kendra can be used for both internal enterprise search (intranet) and external customer-facing search (internet) scenarios.

    Question: What are custom data sources in AWS Kendra?
    Answer: Custom data sources in AWS Kendra allow you to integrate your own data repositories or applications into the search index.

    Question: Can AWS Kendra search across multiple languages?
    Answer: Yes, AWS Kendra supports multiple languages and can perform multilingual search queries.

    Question: How does AWS Kendra handle synonyms and acronyms?
    Answer: AWS Kendra automatically recognizes synonyms and acronyms, improving search accuracy and understanding user queries.

    Question: Does AWS Kendra support natural language processing (NLP)?
    Answer: Yes, AWS Kendra uses natural language processing to understand complex user queries and return relevant search results."""

    tmp = [i for i in test_bank.split("\n") if i != '']
    qa_dict = {}

    for qq, aa in [tmp[i: i + 2] for i in range(0, len(tmp), 2)]:
        qa_dict[qq.split('Question: ')[-1]] = aa.split('Answer: ')[-1]
    return qa_dict

def get_ans_score(ground_truth, generated_answer, model):
    generated_answer_embedding = model.encode(generated_answer, convert_to_tensor=True)
    ground_truth_embedding = model.encode(ground_truth, convert_to_tensor=True)
    return util.pytorch_cos_sim(generated_answer_embedding, ground_truth_embedding)

In [129]:
# Load a pre-trained SentenceTransformer model
model_name = "paraphrase-mpnet-base-v2"
model = SentenceTransformer(model_name)

In [130]:
qa_bank = get_test_bank()

chain = build_chain()

In [133]:
score_dict = {}
for qq, aa in tqdm(qa_bank.items()):
    ans = run_chain(chain, prompt = qq)
    score = get_ans_score(aa, ans['answer'], model=model)
    score_dict[qq] = {'generated_answer': ans['answer'], 'correct_answer': aa, 'score': score}

100%|██████████| 10/10 [01:41<00:00, 10.20s/it]


In [141]:
for kk, vv in score_dict.items():
    vv['score'] = vv['score'].item()
    score_dict[kk] = vv

In [142]:
pd.DataFrame(score_dict)

,What is AWS Kendra?,What types of data sources does AWS Kendra support for indexing?,How does AWS Kendra handle natural language queries?,What are the benefits of using AWS Kendra for enterprise search?,How does AWS Kendra ensure security and compliance?,Can AWS Kendra be used for both internal and external search scenarios?,What are custom data sources in AWS Kendra?,Can AWS Kendra search across multiple languages?,How does AWS Kendra handle synonyms and acronyms?,Does AWS Kendra support natural language processing (NLP)?
generated_answer,AWS Kendra is a search service that helps use...,AWS Kendra supports the following types of da...,AWS Kendra is an intelligent search service t...,"AWS Kendra is a highly scalable, enterprise-g...",AWS Kendra is a cloud-based service that prov...,"Yes, AWS Kendra can be used for both internal...",Custom data sources in AWS Kendra are data so...,"Yes, AWS Kendra can search across multiple la...",AWS Kendra handles synonyms and acronyms by u...,"Yes, AWS Kendra supports natural language pro..."
correct_answer,AWS Kendra is an intelligent search service po...,"AWS Kendra supports various data sources, incl...",AWS Kendra uses machine learning models to und...,Some benefits of AWS Kendra include improved s...,AWS Kendra encrypts data at rest and in transi...,"Yes, AWS Kendra can be used for both internal ...",Custom data sources in AWS Kendra allow you to...,"Yes, AWS Kendra supports multiple languages an...",AWS Kendra automatically recognizes synonyms a...,"Yes, AWS Kendra uses natural language processi..."
score,0.827877,0.766677,0.752259,0.802861,0.766051,0.884344,0.767208,0.872569,0.671585,0.756675


In [148]:
qa_bank_updated = {}

updated_q_list = [
    "Explain the purpose and functionalities of AWS Kendra.",
    "Which data sources are supported by AWS Kendra for indexing, and how does it handle data from these sources?",
    "Describe how AWS Kendra leverages machine learning models to understand and respond to natural language queries effectively.",
    "Discuss the advantages and use cases of using AWS Kendra for enterprise search.",
    "How does AWS Kendra ensure data security and compliance while handling sensitive information?",
    "Can you elaborate on how AWS Kendra can be utilized for both internal enterprise search and external customer-facing search scenarios?",
    "What are custom data sources in AWS Kendra, and how can organizations integrate their data repositories or applications into the search index?",
    "Explain the multilingual capabilities of AWS Kendra and how it handles search queries across different languages.",
    "How does AWS Kendra's advanced recognition of synonyms and acronyms improve the accuracy of search results?",
    "In what ways does AWS Kendra employ natural language processing (NLP) to comprehend complex user queries and provide relevant search results?"
]
for idx, (qq, aa) in enumerate(qa_bank.items()):
    qa_bank_updated[updated_q_list[idx]] = aa

In [149]:
score_dict_updated = {}
for qq, aa in tqdm(qa_bank_updated.items()):
    ans = run_chain(chain, prompt = qq)
    score = get_ans_score(aa, ans['answer'], model=model)
    score_dict_updated[qq] = {'generated_answer': ans['answer'], 'correct_answer': aa, 'score': score}

100%|██████████| 10/10 [01:53<00:00, 11.34s/it]


In [151]:
for kk, vv in score_dict_updated.items():
    vv['score'] = vv['score'].item()
    score_dict_updated[kk] = vv

In [152]:
pd.DataFrame(score_dict_updated)

,Explain the purpose and functionalities of AWS Kendra.,"Which data sources are supported by AWS Kendra for indexing, and how does it handle data from these sources?",Describe how AWS Kendra leverages machine learning models to understand and respond to natural language queries effectively.,Discuss the advantages and use cases of using AWS Kendra for enterprise search.,How does AWS Kendra ensure data security and compliance while handling sensitive information?,Can you elaborate on how AWS Kendra can be utilized for both internal enterprise search and external customer-facing search scenarios?,"What are custom data sources in AWS Kendra, and how can organizations integrate their data repositories or applications into the search index?",Explain the multilingual capabilities of AWS Kendra and how it handles search queries across different languages.,How does AWS Kendra's advanced recognition of synonyms and acronyms improve the accuracy of search results?,In what ways does AWS Kendra employ natural language processing (NLP) to comprehend complex user queries and provide relevant search results?
generated_answer,AWS Kendra is a service that provides a natur...,AWS Kendra supports indexing data from variou...,AWS Kendra leverages machine learning models ...,AWS Kendra is a cloud-based enterprise search...,AWS Kendra ensures data security and complian...,"Yes, AWS Kendra can be utilized for both inte...",Custom data sources in AWS Kendra are a way f...,AWS Kendra is a multilingual search service t...,AWS Kendra's advanced recognition of synonyms...,AWS Kendra employs natural language processin...
correct_answer,AWS Kendra is an intelligent search service po...,"AWS Kendra supports various data sources, incl...",AWS Kendra uses machine learning models to und...,Some benefits of AWS Kendra include improved s...,AWS Kendra encrypts data at rest and in transi...,"Yes, AWS Kendra can be used for both internal ...",Custom data sources in AWS Kendra allow you to...,"Yes, AWS Kendra supports multiple languages an...",AWS Kendra automatically recognizes synonyms a...,"Yes, AWS Kendra uses natural language processi..."
score,0.853952,0.73192,0.835649,0.775592,0.808646,0.843985,0.839747,0.851058,0.840046,0.830578
